A notebook futtatásához be kell állítani a tanításhoz használt forrásképek mappanevét, a style-transferhez szükséges
bemeneti képet, valamint a hiperparaméterek is teszteléshez vannak beállítva.

Ezen kívül az átlagolt képek is egy kezdetleges esetre vannak visszavezetve, az egyszerűség miatt.

In [27]:
train_folder = "train_sample"
filename = "cat.jpg"

In [1]:
from __future__ import print_function
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.layers import Input, Dense, GlobalAveragePooling2D, merge, Dropout, Convolution2D, Lambda
from keras.layers.core import K
import numpy as np
from keras.callbacks import Callback, EarlyStopping
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical
from __future__ import print_function
from keras.preprocessing.image import load_img, img_to_array
from scipy.misc import imsave
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse
##########################################################
# error esetére, elvileg nem okoz gondot 'jó' esetben sem
import tensorflow as tf
tf.python.control_flow_ops = tf
##########################################################



from keras.applications import vgg16

K._LEARNING_PHASE = tf.constant(0)  # test mode

Using TensorFlow backend.


In [2]:
import math
import matplotlib as mp
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import preprocess2 as pp
# a képek betöltése és előfeldolgozása
csv_data, author_stat = pp.csv_load()
# a szükséges alkotókat kiválogatjuk
authors_to_select = author_stat[:]

[('139e4feb4346b2ef87afb7be838e8220', 4),
 ('f258cbc24958b6fd973e711ca4f19354', 5),
 ('7947c26080059d75a1230803dd4307b5', 5),
 ('4f2a6e08d4919a4b7275d9ce6be6fbe3', 5),
 ('2a2f5222c870a3843794188fdfb9a85a', 5),
 ('3267c7910e67fdca840d4c38f72e4b70', 5),
 ('2f5a90d2491aeddfeff7230bf4119f08', 5),
 ('003cb9bdfadc23495b51a630f8af2570', 5),
 ('d357a7e5853194fbd6262a1a8c21bc3f', 5),
 ('8b9d3c0614c9f0f0ac004a4e59872862', 5),
 ('0d899a0eb3988f433ffc667b55bd430e', 5),
 ('07c533c28369f1e2b027f6a0c827d90e', 5),
 ('0c4aebb696abe69e8b437abba61f6084', 5),
 ('513530a8e473ab2f1d89fcf3684829d4', 5),
 ('5aa46c4bd5cd785c39b45c221984838e', 6),
 ('ad7236b81e5471a18f7a7b8a2201ea88', 6),
 ('bbeaac68870859dddc0fc669cbd7734c', 6),
 ('bf4915402893875e357cc537fd89dd9f', 6),
 ('093d54c8398df0ab85b48e53faa54940', 6),
 ('294d0a61d897faf13708824d9a86bb04', 6),
 ('f26d55e95372a879c9ec4f7a91e19076', 6),
 ('37c963182bf5c9a5bf9504bb18d2a669', 6),
 ('6be4b4c5a63d2508f0fc06d7dd8ad391', 6),
 ('2574d646c4252bdd87d253847eedfdf

In [4]:
csv_data = pp.csv_select(csv_data, authors_to_select)
# az útvonalnak az összes képet tartalmazó könyvtárra kell mutatnia, mert különben
# nem fog tudni belőle elég képet kiolvasni
train_images, labels = pp.load_images(csv_data, train_folder)

14 images in the result


In [5]:
# adatok megfelelő formátumra hozása a keras számára
encoder = preprocessing.LabelEncoder()
encoder.fit(labels)

encoded_l = encoder.transform(labels)
#print(encoded_l)

labels_onehot = to_categorical(encoded_l)
#print(labels_onehot)

[ 3  7  4 13  9  8  2  1 12  5  6  0 11 10]
[[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]]


In [6]:
#  Ezt az osztályt használjuk fel az eredmények megjelenítésére

class TrainingHistory(Callback):
    # Tanulási folyamat elején létrehozunk egy-egy üres listát a kinyerni kívánt metrikák tárolása céljából.
    def on_train_begin(self, logs={}):
        # Hiba mértéke a tanító adatokon.
        self.losses = []
        # Hiba mértéke a validációs adatokon.
        self.valid_losses = []
        # A modell jóságát, pontosságát mérő mutatószám a tanító adatokon. 
        self.accs = []
        # A modell jóságát, pontosságát mérő mutatószám a validációs adatokon. 
        self.valid_accs = []
        # A tanítási fázisok sorszámozása.
        self.epoch = 0
    
    # Minden egyes tanítási fázis végén mentsük el, hogy hogyan teljesít aktuálisan a háló. 
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 1 == 0:
            self.losses.append(logs.get('loss'))
            self.valid_losses.append(logs.get('val_loss'))
            self.accs.append(logs.get('acc'))
            self.valid_accs.append(logs.get('val_acc'))
            self.epoch += 1
            
history = TrainingHistory()

In [7]:
# Beállítjuk a szükséges konstansokat
img_width = 256
img_height = 256
img_nrows = img_width
img_ncols = img_height

# A súlyozással lehet beállítani a stílus és a tartalom arányát
total_variation_weight = 1.
style_weight = 1.
content_weight = 0.025

In [8]:
# a képfeldolgozáshoz szükséges utility függvények

def random_image():
    img = np.random.random((256,256,3))
    return img

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_width, img_height))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg16.preprocess_input(img)
    return img

def deprocess_image(x):
    x = x.reshape((img_width, img_height, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

img_size = (img_width, img_height, 3)

In [9]:
# Ebben a tanzorban fogjuk tárolni a háló bemenetéhez az adatokat
dream = Input(batch_shape=(None,) + (256,256,3))

In [10]:
# előtanított modell betöltése, a fully-connected rétegek nélkül
base_model = vgg16.VGG16(include_top=False, weights='imagenet', input_tensor=dream)

In [11]:
# kinyerjük a stílusjegyeket a cnn köztes rétegegeiből (és max pool cnn kimeneti rétegére)
#desired_layers = [28, 44, 60, 70, 92, 114, 136, 158, 172, 194]
desired_layers = [11,12,13] # a VGG16-nál ezek a rétegek kellettek
style_layers = []

base_style = GlobalAveragePooling2D()(base_model.output)
base_style = Dense(base_model.output_shape[3], activation='relu')(base_style)

style_layers += [base_style]

for i in range(len(desired_layers)):
    sl = base_model.layers[desired_layers[i]].output
    sl = GlobalAveragePooling2D()(sl)
    sl = Dense(base_model.layers[desired_layers[i]].output_shape[3], activation='relu', name=('style_' + str(i)))(sl)
    style_layers += [sl]

In [12]:
# egymás mellé tesszük a különböző szintű feature-öket
style_output = merge(style_layers, mode='concat', name='style_output')

# ezután hozzáadunk két előrecsatolt réteget ReLU aktivációs függvénnyel
ff = Dense(1024, activation='relu')(style_output)
ff = Dropout(0.5)(ff)
ff = Dense(512, activation='relu')(ff)

# és végül egy kimenete lesz a hálónak - a "binary_crossentropy" költségfüggvénynek erre van szüksége
predictions = Dense(labels_onehot.shape[1], activation='softmax')(ff)
# a model létrehozása
model = Model(input=base_model.input, output=predictions)

In [13]:
# két lépésben fogjuk tanítani a hálót
# az első lépésben csak az előrecsatolt rétegeket tanítjuk, a konvolúciós rétegeket befagyasztjuk
for layer in base_model.layers:
    layer.trainable = False

early_stopping = EarlyStopping(monitor='val_loss', patience=5)
# lefordítjuk a modelt (fontos, hogy ezt a rétegek befagyasztása után csináljuk"
# mivel két osztályunk van, ezért bináris keresztentrópia költségfüggvényt használunk
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'], callbacks=[early_stopping])

In [14]:
model.fit(train_images, labels_onehot, batch_size=8, nb_epoch=1, validation_split=0.2, callbacks=[history])

/home/kohlmann/anaconda3/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:1107: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Train on 11 samples, validate on 3 samples
Epoch 1/1
11/11 [==============================] - 16s - loss: 14.6529 - acc: 0.0909 - val_loss: 16.1181 - val_acc: 0.0000e+00


In [15]:
# másodjára már itt is tanítunk
for layer in base_model.layers:
    layer.trainable = True

# lefordítjuk a modelt (fontos, hogy ezt a rétegek befagyasztása után csináljuk"
# mivel több osztályunk van, ezért kategórikus keresztentrópia költségfüggvényt használunk
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'], callbacks=[early_stopping])

In [16]:
model.fit(train_images, labels_onehot, batch_size=8, nb_epoch=1, validation_split=0.2, callbacks=[history])

/home/kohlmann/anaconda3/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:1107: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Train on 11 samples, validate on 3 samples
Epoch 1/1
11/11 [==============================] - 39s - loss: 14.6528 - acc: 0.0909 - val_loss: 16.1181 - val_acc: 0.0000e+00


In [17]:
# köztes adatok a predikciókhoz
predicted_values = model.predict(np.asarray(train_images))

/home/kohlmann/anaconda3/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:1107: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


In [18]:
# köztes adatok a predikciókhoz
max_pred = np.zeros(predicted_values.shape, dtype=int)
festok = []
for i, pred in enumerate(predicted_values):
    d = np.argmax(pred)
    max_pred[i, d] = 1
    festok.append(d)
#festok

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [19]:
encoder.inverse_transform(encoded_l)

array(['5b6a7be5ffc6a27b91bd3210ffa2e088',
       'cc47068929413a16aa707faefbdf4b70',
       '69904cf890070e9593a566394d5dece4',
       'f44205b1eb2de981de766e0688f8cbac',
       'da9ab2081b197129eeb91477d239be00',
       'd500fe452aef7a6f90de16197a9670bf',
       '485d901dc4df30b128bf01cb6e229767',
       '3f93b217bd0dbf874f973958f1eb6df4',
       'f14a3a6cc3112c9e92bc6c33c88eb264',
       '8e441c5899bf3d2f3b2c493e62fb92bf',
       'c56bcab4b317984013ebef5d3c4b5906',
       '3cc9a44380296d93e68b71a27643c25f',
       'f0a20221a0109091e49d14c761574cd8',
       'e1587900e782de448f604b37cde0fdfd'], 
      dtype='<U32')

In [20]:
# egy szótárban tároljuk a rétegek kimeneteit
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

In [21]:
# Itt válasszuk ki, hogy melyik képek stílusait akarjuk akkumulálni
# Tesztelési célból az most az első két kép, de természetesen alkalmazás közben
# ez az egy festő képeinek halmaza
desired_style_images = train_images[0:1]

In [22]:
# Itt gyakorlatilag a kívánt képeket végigvezetjük a hálón, és átlagoljuk a hozzájuk
# tartozó stíluskimeneteket, amelyek a style transferhez fognak kelleni.
feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
style_outputs = {}
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    get_style_layer_output = Model(dream, layer_features)
    sum_output = None
    layer_outputs = get_style_layer_output.predict(desired_style_images)
    for image in layer_outputs:
        if sum_output == None:
            sum_output = image
        else:
            sum_output += image
    sum_output /= len(desired_style_images)
    style_outputs[layer_name] = sum_output

In [26]:
# Itt pedig maga a style transfer található, az eredetihez képest minimális módosítással
img_nrows = 256
img_ncols = 256
assert img_ncols == img_nrows, 'Due to the use of the Gram matrix, width and height must match.'

# a bemeneti képünkbül készített tensor
base_image = K.variable(preprocess_image(filename))
# eredetileg ezt használtuk volna a style transferhez:
style_reference_image = K.variable(preprocess_image('style.jpg'))

# egy kombinált tenzort is létrehozunk, amelyet a hálón vezetünk végig
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

# és ebbe kombináljuk az előzőket.
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

# mivel a súlyokat már kiátlagoltuk, a style transferre már használhatjuk a hagyományos modellt
model = vgg16.VGG16(input_tensor=input_tensor, weights='imagenet', include_top=False)
print('Model loaded.')

# the gram matrix a stílus veszteséghez
def gram_matrix(x):
    #assert K.ndim(x) == 3
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# amit itt fel is használunk
def style_loss(style, combination):
    #assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

# a tartalomért felelős veszteségfüggvény
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

# a generált kép lokális koherenciájának megőrzésére szolgáló veszteség a harmadik
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(x[:, :img_nrows-1, :img_ncols-1, :] - x[:, 1:, :img_ncols-1, :])
    b = K.square(x[:, :img_nrows-1, :img_ncols-1, :] - x[:, :img_nrows-1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

# a rétegek kimenetei
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

# a veszteséget egyetlen skalárba kombináljuk
loss = K.variable(0.)
layer_features = outputs_dict['block4_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(base_image_features,
                                      combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    # itt az eredeti megoldás helyett az átlagolt stílust adjuk bemenetként
    # a stílusfeszteség függvényének
    #style_reference_features = layer_features[1, :, :, :]
    style_reference_features = style_outputs[layer_name]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(feature_layers)) * sl
loss += total_variation_weight * total_variation_loss(combination_image)

# majd kiszámoljuk a gradienst
grads = K.gradients(loss, combination_image)

outputs = [loss]
if type(grads) in {list, tuple}:
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# ezzel az osztályjal számoljuk ki egy függvényben
# hatékonyan a veszteséget valamint a gradienst is
class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

# scipy-alapú optimalizációt (L-BFGS) használunk a kép pixeleire
# hogy minimalizáljuk a nerális stílusveszteséget
x = np.random.uniform(0, 255, (1, img_nrows, img_ncols, 3)) - 128.

for i in range(1):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # elmentjük az aktuálisan generált képet
    img = deprocess_image(x.copy())
    fname = 'cat3_at_iteration_%d.png' % i
    imsave(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Model loaded.
Start of iteration 0
Current loss value: 1.77374e+10
Image saved as cat3_at_iteration_0.png
Iteration 0 completed in 142s
